# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [100]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [101]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
filepath = "data/song_data"

In [5]:
song_files = get_files(filepath)

In [6]:
song_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [7]:
df = pd.read_json(song_files[0], typ='series')
df

num_songs                            1
artist_id           ARD7TVE1187B99BFB1
artist_latitude                   None
artist_longitude                  None
artist_location        California - LA
artist_name                     Casual
song_id             SOMZWCG12A8C13C480
title                 I Didn't Mean To
duration                       218.932
year                                 0
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [17]:
song_data = list(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values)
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [18]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [19]:
df

num_songs                            1
artist_id           ARD7TVE1187B99BFB1
artist_latitude                   None
artist_longitude                  None
artist_location        California - LA
artist_name                     Casual
song_id             SOMZWCG12A8C13C480
title                 I Didn't Mean To
duration                       218.932
year                                 0
dtype: object

In [8]:
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values)
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [8]:
filepath = "data/log_data"

In [9]:
log_files = get_files(filepath)

In [27]:
log_files[0]

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [14]:
df = pd.read_json(log_files[0], lines=True)
df.head()
# log_files[0]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [102]:
from datetime import datetime as dt

pd.options.mode.chained_assignment = None

# Filter NextSong action
df_time = df[df.page == "NextSong"]

# Convert `ts` to timestamp
df_time['ts'] = df_time['ts'].apply(lambda x: dt.fromtimestamp(x/1000.0))

In [47]:
# df_time.ts.dt.hour
# df_time.ts.dt.day
# df_time.ts.dt.week
# df_time.ts.dt.month
# df_time.ts.dt.year
df_time.ts.dt.weekday

0      4
1      4
2      4
3      4
4      4
5      4
6      4
7      4
9      4
10     4
11     4
12     4
13     4
14     4
15     4
16     4
17     4
18     4
19     4
23     4
24     4
25     4
26     4
27     4
30     4
31     4
33     4
35     4
36     4
38     4
      ..
356    4
357    4
358    4
359    4
360    4
361    4
362    4
363    4
364    4
365    4
366    4
367    4
368    4
369    4
370    4
371    4
372    4
373    4
374    4
375    4
376    4
377    4
378    4
380    4
381    4
382    4
383    4
384    4
385    4
387    4
Name: ts, Length: 330, dtype: int64

In [103]:
time_data = (df_time.ts,
             df_time.ts.dt.hour,
             df_time.ts.dt.day,
             df_time.ts.dt.week,
             df_time.ts.dt.month,
             df_time.ts.dt.year,
             df_time.ts.dt.weekday)
column_labels = ('timestamp', 'hour', 'day', 'week of year', 'month', 'year', 'weekday')

In [105]:
time_df = pd.concat(time_data, axis=1, keys=column_labels)
time_df.head()

,timestamp,hour,day,week of year,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [106]:
for i, row in time_df.iterrows():
    print(list(row))
    cur.execute(time_table_insert, list(row))
    conn.commit()

[Timestamp('2018-11-30 00:22:07.796000'), 0, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:08:41.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:12:48.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:17:05.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:20:56.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:24:32.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:28:19.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:32:00.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:38:17.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:39:53.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:44:10.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:48:36.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:50:28.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:55:35.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 02:01:08.796000'), 2, 30, 48, 11, 2018, 4]
[Timestamp

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [99]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.